In [ ]:
import os
from cognite.client import CogniteClient
from getpass import getpass
project = "publicdata"
api_url = f"/api/playground/projects/{project}/context/pnidobjects"
api_key = getpass("Open Industrial Data API-KEY: ")
c = CogniteClient(api_key=api_key,
                       project=project, client_name="pnid demo")

## Import utils for demo purpose only

In [ ]:
!pip install pdf2image
!pip install bounding-box
from PIL import Image
import numpy as np
from pdf2image import convert_from_path
from bounding_box import bounding_box as bb
import json
import time
import pprint
pp = pprint.PrettyPrinter(indent=2)

def draw_bbox(pnid_img, result):
    img_arr = np.array(pnid_img)
    height, width = img_arr.shape[:-1]
    img_arr_copy = img_arr[:,:,::-1].copy()
    for obj in result:
        bbox = obj["boundingBox"]
        label = obj["type"]
        bb.add(
            img_arr_copy,
            int(bbox["xMin"]*width), 
            int(bbox["yMin"]*height),
            int(bbox["xMax"]*width),
            int(bbox["yMax"]*height),
            label,
            "red"
        )
    return Image.fromarray(img_arr_copy[:,:,::-1])

## Download the P&ID to visualize

In [ ]:
file_id = 230063753840368
res = c.files.retrieve(id=file_id)
print(res.name)
c.files.download_to_path(f"./{res.name}.pdf", id=res.id)

In [ ]:
pnid_img = convert_from_path(f"./{res.name}.pdf")[0]
pnid_img

## /findobjects endpoint

In [ ]:
findobjects_url = f"{api_url}/findobjects"
post_body = {"fileId": file_id}
res = c.post(url = findobjects_url, json = post_body)
job_id = json.loads(res.text)["jobId"]
print("jobId:", job_id)

In [ ]:
%%time
res = c.get(url = f"{api_url}/{job_id}")

while json.loads(res.text)["status"] not in ["Completed", "Failed"]:
    time.sleep(5)
    print(json.loads(res.text)["status"])
    res = c.get(url = f"{api_url}/{job_id}")


result = json.loads(res.text)["items"]
print(f"found {len(result)} objects")
print("The first object parameters:")
pp.pprint(result[0])

In [ ]:
draw_bbox(pnid_img, result)

## /findsimilar endpoint

In [ ]:
template ={
    'boundingBox': {
        'xMax': 0.5839555757026292,
        'xMin': 0.5684496826835902,
        'yMax': 0.2116289012398461,
        'yMin': 0.1911073108165883},
    'type': 'shared indicator'}
x_max, x_min, y_max, y_min = template["boundingBox"].values()
width, height = pnid_img.size
pnid_img.crop((
    int(x_min*width),
    int(y_min*height),
    int(x_max*width),
    int(y_max*height)
))

In [ ]:
findsimilar_url = f"{api_url}/findsimilar"
post_body = {
    "fileId": file_id,
    "template": template
}
res = c.post(url = findsimilar_url, json = post_body)
job_id = json.loads(res.text)["jobId"]
print("jobId:", job_id)

In [ ]:
%%time
res = c.get(url = f"{api_url}/{job_id}")

while json.loads(res.text)["status"] not in ["Completed", "Failed"]:
    time.sleep(1)
    print(json.loads(res.text)["status"])
    res = c.get(url = f"{api_url}/{job_id}")


result = json.loads(res.text)["items"]
print(f"found {len(result)} objects")
print("The first object parameters:")
pp.pprint(result[0])

In [ ]:
draw_bbox(pnid_img, result)